# 类的方法

## 方法（Method）
---

In [3]:
class MyClass(object):
    i = 1
    def foo(self):
        pass
mc = MyClass()
mc.foo()

上面定义了一个类`MyClass`，类中定义了一个方法`foo`,如果像函数一样对`foo`自身的调用都会失败：

In [2]:
foo()

NameError: name 'foo' is not defined

直接调用引发了NameError的异常，因为在全局函数中没有这样的函数存在。也就是说`foo`是一个方法表示它属于一个类而不是全局空间中的名字。

In [4]:
MyClass.foo()

TypeError: foo() missing 1 required positional argument: 'self'

In [5]:
MyClass.i

1

直接调用`MyClass.foo()`会抛出TypeError的异常，这是因为为了与OOP惯例保持一致，Python严格要求：没有实例的方法是不能被调用的。这种限制即Python所描述的绑定概念（binding），在此，**方法必须绑定（到一个实例）才能直接被调用**。

## 绑定和方法调用
---

+ 方法仅仅是类内部定义的函数（这意味着方法是类属性而不是实例属性）。
+ 方法只有在其所属的类拥有实例时才能够被调用，当存在一个实例时，方法才被认为是绑定到那个实例了，否则就是未绑定的。
+ 任何一个方法定义中的第一个参数都是`self`，他表示调用此方法的实例对象

### 调用绑定方法